In [ ]:
import cv2
import numpy as np
import os
import imageio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def simplest_cb(img, percent):
    out_channels = []
    # print(img.shape)
    # (480, 640, 3)

    # split the image into its three channels
    # blue, green, red = cv2.split(img)
    channels = cv2.split(img)
    # print(channels[0].shape)
    # (480, 640)

    totalstop = channels[0].shape[0] * channels[0].shape[1] * percent / 200.0
    # print(totalstop)

    for channel in channels:
        # use the cv2.calcHist function to build grayscale histograms
        # cv2.calcHist(images, channels, mask, histSize, ranges)
        """ images: This is the image that we want to compute a histogram for. Wrap it as a list: [myImage] 
            channels: A list of indexes, where we specify the index of the channel we want to compute a histogram for. 
                    To compute a histogram of a grayscale image, the list would be [0]
                    histogram for all three red, green, and blue channels, the channels list would be [0, 1, 2]
            mask: here we can supply a mask. If a mask is provided, a histogram will be computed for masked pixels only.
                If we do not have a mask or do not want to apply one, we can just provide a value of None
            histSize: This is the number of bins we want to use when computing a histogram [256]
            ranges: The range of possible pixel values. Normally, this is (0, 256)
        """
        bc = cv2.calcHist([channel], [0], None, [256], (0, 256), accumulate=False)

        # np.cumsum(bc) : Return the cumulative sum of the elements in bc(input array).

        # low value
        lv = np.searchsorted(np.cumsum(bc), totalstop)
        # print(lv)

        # high value
        hv = 255 - np.searchsorted(np.cumsum(bc[::-1]), totalstop)
        # print(hv)

        lut = np.array([0 if i < lv else (255 if i > hv else round(float(i - lv) / float(hv - lv) * 255)) for i in
                        np.arange(0, 256)], dtype="uint8")

        out_channels.append(cv2.LUT(channel, lut))

    # join individual channels to create a BGR image
    return cv2.merge(out_channels)

In [ ]:
#in_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/train-set/raw'
#out_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/train-set/cb_raw'

in_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/raw'
out_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/cb_raw'

for imagePath in os.listdir(in_path):
    input_path = os.path.join(in_path, imagePath)
    img = imageio.imread(input_path)
    result = simplest_cb(img, 1)
    full_out_path = os.path.join(out_path, imagePath)
    cv2.imwrite(full_out_path, result)

In [ ]:
"""


import numpy as np
import cv2
import os

from sklearn.metrics import mean_squared_error

target_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/reference'
predicted_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/cb_raw'

target_data = []
predict_data = []


im_h = 112
im_w = 112
for image_name in os.listdir(target_path):

    #read the raw images
    target_img = cv2.imread(os.path.join(target_path, image_name))
    predicted_img = cv2.imread(os.path.join(predicted_path, image_name))
    

    #resize the images into 112 x 112
    target_img = cv2.resize(target_img, (im_h, im_w))
    predicted_img = cv2.resize(predicted_img, (im_h, im_w))
        
    # put the images into array list
    target_data.append(target_img)
    predict_data.append(predicted_img)


# convert arrays into numpy arrays
target_data = np.array(target_data)
predict_data = np.array(predict_data)

print(target_data.shape, predict_data.shape)


%cd '/content/drive/MyDrive/UnderwaterImageEnhancement/models/cb-raw-predicted/'

# save the numpy arrays
np.save('target_data', target_data)
np.save('predict_data', predict_data)


"""

In [ ]:
"""


%cd '/content/drive/MyDrive/UnderwaterImageEnhancement/models/cb-raw-predicted/'

# load the numpy arrays
target_data = np.load('target_data.npy')
predict_data = np.load('predict_data.npy')

print(target_data.shape, predict_data.shape)


"""

In [ ]:
"""

import numpy as np
  
# Mean Squared Error
MSE = np.square(np.subtract(target_data,predict_data)).mean()
print("MSE for test data :",MSE)


"""

In [ ]:
"""

mse = (np.square(target_data - predict_data)).mean(axis=None)

print("MSE for test data :",mse)

""""

In [ ]:
"""


from math import log10, sqrt

def PSNR(target_data, predict_data):
 mse = np.mean((target_data - predict_data) ** 2)
 print(mse)
 if(mse == 0):
  return 100
 max_pixel = 255.0
 psnr = 20 * log10(max_pixel / sqrt(mse))
 return psnr 


value = PSNR(target_data, predict_data)
print("\nPeak Signal-to-Noise Ratio (PSNR) value is", value, "dB")


"""

In [ ]:
"""

from skimage.metrics import structural_similarity as ssim

ssim_value = ssim(target_data, predict_data, multichannel=True)
print("SSIM:", ssim_value)

"""

In [ ]:
"""

from skimage.measure import compare_ssim

(score, diff) = compare_ssim(target_data, predict_data, full=True, multichannel=True)
diff = (diff * 255).astype("uint8")
print("SSIM:", score)


"""

In [ ]:
###### FULL-REFERENCE IMAGE QUALITY ASSESSMENT IN TERMS OF MSE, PSNR, AND SSIM #######

import numpy as np
import cv2
import os

from skimage.metrics import mean_squared_error
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity 


target_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/reference'
predicted_path = '/content/drive/MyDrive/UnderwaterImageEnhancement/dataset/test-set/cb_raw'


im_h = 224
im_w = 224

sumMSE = 0;
sumPSNR = 0;
sumSSIM = 0;
count = 0;


for image_name in os.listdir(target_path):

    #read the raw images
    target_img = cv2.imread(os.path.join(target_path, image_name))
    predicted_img = cv2.imread(os.path.join(predicted_path, image_name))
    

    #resize the images into 224 x 224
    target_img = cv2.resize(target_img, (im_h, im_w))
    predicted_img = cv2.resize(predicted_img, (im_h, im_w))


    #calculate the MSE, PSNR, and SSIM 
    MSE = mean_squared_error(target_img, predicted_img)
    PSNR = peak_signal_noise_ratio(target_img, predicted_img)
    SSIM = structural_similarity(target_img, predicted_img, multichannel=True)

    sumMSE = sumMSE + MSE
    sumPSNR = sumPSNR + PSNR
    sumSSIM = sumSSIM + SSIM
    count = count + 1
    
    # print('SSIM: ', SSIM)
    # print('MSE: ', MSE)
    # print('PSNR: ', PSNR)


# print('sumMSE: ',sumMSE)
# print('sumPSNR: ', sumPSNR)
# print('sumSSIM: ', sumSSIM)
# print('count: ', count)


# Get the Average value 
mse_value = sumMSE/count
psnr_value = sumPSNR/count
ssim_value = sumSSIM/count


print("MSE for testing data :", format(mse_value, ".4f"))
print("PSNR for testing data :", format(psnr_value, ".4f"), "dB")
print("SSIM for testing data :", format(ssim_value, ".4f"))

MSE for testing data : 4027.9611
PSNR for testing data : 13.3992 dB
SSIM for testing data : 0.7208
